# 🎧 Spotify Data Monitor & Analytics
**Responsável:** Matheus </br>
**Status:** 🚧 Em Desenvolvimento (Fase Bronze) </br>
**Última Atualização:** 2026-02-03 </br>

---

## 🎯 Objetivo 
Pipeline de engenharia de dados para processar o histórico estendido de streaming do Spotify (GDPR Data), separando Áudio e Vídeo, para monitoramento de hábitos de consumo.

## 📑 Índice
1. [Setup e Bibliotecas](#setup-e-bibliotecas)
2. [Definição do Database](#definição-do-database)
3. [Camada Bronze: Ingestão Unificada](#camada-bronze-ingestão-unificada)
4. [Validação da Carga](#validação-da-carga)

---

## ⚙️ Configurações iniciais

In [0]:
%pip install openpyxl

In [0]:
from pyspark.sql.functions import current_timestamp, input_file_name,col, sum as _sum
import pandas as pd
import json
import time

In [0]:
PATH_ORIGEM = "/Volumes/sandbox_prd/raw_layer/files/spotify/me/extended/Streaming_History_*.json"
NOME_TABELA_DESTINO = "sandbox_prd.bronze_layer.streaming_history_user_spotify"

## 🥉 Bronze Layer
Leitura dos arquivos JSON e gravação em tabela Delta.

In [0]:
# 1. Leitura (Inferindo Schema)
df_input = (spark.read
            .format("json")
            .option("multiline", "true") 
            .option("inferSchema", "true") 
            .load(PATH_ORIGEM))

# 2. Enriquecimento (Usando _metadata do Unity Catalog)
# AQUI ESTÁ A CORREÇÃO: Usamos col("_metadata.file_path") em vez da função antiga
df_bronze = df_input.select(
    "*", 
    current_timestamp().alias("dt_ingestion"), 
    col("_metadata.file_path").alias("source_file") 
)

# 3. Gravação (Schema Evolution)
(df_bronze.write
    .format("delta")
    .mode("overwrite")              
    .option("mergeSchema", "true")  
    .saveAsTable(NOME_TABELA_DESTINO)
)

print(f"✅ Carga concluída com sucesso em: {NOME_TABELA_DESTINO}")
display(df_bronze.limit(5))

In [0]:
# 1. Leitura dos arquivos JSON (Inferindo Schema)
df_input = (spark.read
            .format("json")
            .option("multiline", "true") 
            .option("inferSchema", "true") 
            .load(PATH_ORIGEM))

In [0]:
# 2. Enriquecimento dos dados com metadados do Unity Catalog
df_bronze = df_input.select(
    "*", 
    current_timestamp().alias("dt_ingestion"), 
    col("_metadata.file_path").alias("source_file") 
)

In [0]:
# 3. Gravação dos dados na tabela Delta (Schema Evolution)
(df_bronze.write
    .format("delta")
    .mode("overwrite")              
    .option("mergeSchema", "true")  
    .saveAsTable(NOME_TABELA_DESTINO)
)

print(f"✅ Carga concluída com sucesso em: {NOME_TABELA_DESTINO}")

In [0]:
# Exibe uma amostra dos dados carregados
display(df_bronze.limit(5))

## 🔍 Quality Check

Query rápida para conferir os dados.

In [0]:
%sql
-- Validação 1: Volume de dados por arquivo de origem
SELECT 
    source_file,
    count(*) as total_linhas
FROM sandbox_prd.bronze_layer.streaming_history_user_spotify
GROUP BY source_file
ORDER BY source_file

In [0]:
%sql
-- Validação 2: Período dos dados (Min e Max)
SELECT 
    min(ts) as primeira_reproducao,
    max(ts) as ultima_reproducao,
    count(*) as total_geral
FROM sandbox_prd.bronze_layer.streaming_history_user_spotify

In [0]:
# Validação 3: Verificando consistência dos campos principais
df_check = spark.read.table("sandbox_prd.bronze_layer.streaming_history_user_spotify")

# Conta quantos nulos existem nas colunas chave
df_check.select(
    _sum(col("master_metadata_track_name").isNull().cast("int")).alias("nulos_track_name"),
    _sum(col("master_metadata_album_artist_name").isNull().cast("int")).alias("nulos_artist_name"),
    _sum(col("ts").isNull().cast("int")).alias("nulos_timestamp")
).display()

## 🥈 Silver Layer

Tratamento dos dados

In [0]:
%sql
--desc sandbox_prd.bronze_layer.streaming_history_user_spotify
select * from sandbox_prd.bronze_layer.streaming_history_user_spotify

In [0]:
# Lê a base bronze
df_bronze = spark.read.table("sandbox_prd.bronze_layer.streaming_history_user_spotify")

# Listas colunas para remover 
drop_columns = [
    "audiobook_chapter_title",
    "audiobook_chapter_uri",
    "audiobook_title",
    "audiobook_uri",
    "conn_country",
    "incognito_mode",
    "ip_addr",
    "dt_ingestion",
    "source_file"
]

# Remove as colunas desnecessárias
df_silver = df_bronze.drop(*drop_columns)

In [0]:
['episode_name', 'episode_show_name', 'master_metadata_album_album_name', 'master_metadata_album_artist_name', 'master_metadata_track_name', 'ms_played', 'offline', 'offline_timestamp', 'platform', 'reason_end', 'reason_start', 'shuffle', 'skipped', 'spotify_episode_uri', 'spotify_track_uri', 'ts']
# Altera o nome das colunas
df_silver = df_silver.select(
    col("").alias("")
)
